<a href="https://colab.research.google.com/github/rulas99/thermal_anomaly_detection/blob/main/ChooseAndSplitData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from glob import glob
import numpy as np
import cv2
from tqdm import tqdm
from pandas import DataFrame
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
label_imgs = glob('/content/drive/MyDrive/segment_sentinel2_hotspot/mask_correction/*')

In [5]:
def check_label(path):
  img = cv2.imread(path)
  return f'{path}-yes' if 255 in img else f'{path}-no'

In [6]:
with Pool(4) as p:
  res = p.map(check_label, label_imgs)

In [7]:
data = list(zip(*[i.split('-') for i in res]))

In [8]:
df = DataFrame({'label_path':data[0],'anomaly':data[1]})

In [9]:
df.to_csv('/content/drive/MyDrive/segment_sentinel2_hotspot/images_info.csv',index=False)

In [10]:
t_df = df[df.anomaly=='yes']

In [11]:
t_df.shape

(509, 2)

In [12]:
train, test = train_test_split(t_df, test_size=.03, train_size=.97, random_state=99)

In [13]:
len(train), len(test)

(493, 16)

In [23]:
train_names = [f'{i.split("/")[-1][:-8]}B12B11B8A.png' for i in train.label_path]
test_names = [f'{i.split("/")[-1][:-8]}B12B11B8A.png' for i in test.label_path]

In [25]:
thermal_imgs_path = '/content/drive/MyDrive/segment_sentinel2_hotspot/classify_sentinel2_trainingdataset'

In [26]:
train['ori_path'] = [f'{thermal_imgs_path}/{i}' for i in train_names]
test['ori_path'] = [f'{thermal_imgs_path}/{i}' for i in test_names]

In [28]:
train.to_csv('/content/drive/MyDrive/segment_sentinel2_hotspot/train_2.csv',index=False)
test.to_csv('/content/drive/MyDrive/segment_sentinel2_hotspot/test_2.csv',index=False)